## ResNext 29 4x64

 - Training Dataset:  RandAugment, N=3, M=20
   Validation with Unaugmented Data
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [4]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [5]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


#### Train the Model

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnext.yaml \
    model.resnext.cardinality 4 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_3_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00 \
    scheduler.epochs 400

[2020-07-17 12:10:28] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_3_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck

#### Refine the Model

In [ ]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    model.resnext.cardinality 4 \
    train.batch_size 128 \
    train.base_lr 0.001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-07-19 21:44:10] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_t

#### Evaluate Test Datasets

In [6]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00_resume400_50/test_results_0050_cifar10

[2020-07-20 23:38:29] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:36<00:00,  2.19it/s]
[2020-07-20 23:39:06] __main__ INFO: Elapsed 36.11
[2020-07-20 23:39:06] __main__ INFO: Loss 0.3331 Accuracy 0.9027


In [7]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00_resume400_50/test_results_0050_cifar101

[2020-07-20 23:39:26] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.13it/s]
[2020-07-20 23:39:34] __main__ INFO: Elapsed 7.51
[2020-07-20 23:39:34] __main__ INFO: Loss 0.6909 Accuracy 0.8175


In [8]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/test_results_0400_cifar10

[2020-07-20 23:39:47] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:36<00:00,  2.18it/s]
[2020-07-20 23:40:24] __main__ INFO: Elapsed 36.20
[2020-07-20 23:40:24] __main__ INFO: Loss 1.5333 Accuracy 0.6144


In [9]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/checkpoint_00400.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/test_results_0400_cifar101

[2020-07-20 23:40:43] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.14it/s]
[2020-07-20 23:40:51] __main__ INFO: Elapsed 7.49
[2020-07-20 23:40:51] __main__ INFO: Loss 1.9963 Accuracy 0.5025


#### Record the Results

In [10]:
import pandas as pd
model = 'resnext_29_4x64d_ra_3_20_c10val'
model_refined = model + '_refined400'

a = pd.Series([model, 400, 'cifar10', 1.5333, 0.6144])
c = pd.Series([model, 400, 'cifar10.1', 1.9963, 0.5025])

e = pd.Series([model_refined, 50, 'cifar10.1', 0.6909, 0.8175])
f = pd.Series([model_refined, 50, 'cifar10', 0.3331, 0.9027])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 96.4 if row[2] == 'cifar10' else 89.6), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (96.0, 96.7) if row[2] == 'cifar10' else (88.2, 90.9)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/' + model + '/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnext_29_4x64d_ra_3_20_c10val,400,cifar10,1.5333,0.6144,96.4,"(96.0, 96.7)"
1,resnext_29_4x64d_ra_3_20_c10val,400,cifar10.1,1.9963,0.5025,89.6,"(88.2, 90.9)"
2,resnext_29_4x64d_ra_3_20_c10val_refined400,50,cifar10.1,0.6909,0.8175,89.6,"(88.2, 90.9)"
3,resnext_29_4x64d_ra_3_20_c10val_refined400,50,cifar10,0.3331,0.9027,96.4,"(96.0, 96.7)"


In [11]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/resnext_29_4x64d_ra_3_20_c10val'
path = '/home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20_c10val'

s3_resource = boto3.resource("s3", region_name="us-east-1")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)